## <span style="color:#ff5f27">📝 Imports </span>

In [15]:
!pip install -q hopsworks rdflib sentence-transformers pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.9/765.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import hopsworks

from sentence_transformers import SentenceTransformer
from hsfs.embedding import EmbeddingIndex

from functions.zotero_parser import ZoteroRDFParser
from functions.PDF_extractor import ContentProcessor


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## <span style="color:#ff5f27"> Global Config

In [ ]:

# PROJECT_NAME = "你的Hopsworks项目名"

# ZOTERO_ROOT = "/content/ID2223-Project/zotero"

# RDF_PATH = f"{ZOTERO_ROOT}/location privacy.rdf"
# BASE_DIR = ZOTERO_ROOT

# MODEL_NAME = "all-MiniLM-L6-v2"

# MIN_FULLTEXT_LEN = 100

In [ ]:
import os
from config import (
    RDF_PATH,
    BASE_DIR,
    EMBEDDING_MODEL_NAME,
    MIN_FULLTEXT_LEN,
    HOPSWORKS_PROJECT,
    HOPSWORKS_API_KEY,
    META_FG_NAME,
    META_FG_VERSION,
    FULLTEXT_FG_NAME,
    FULLTEXT_FG_VERSION,
    META_FV_NAME,
    FULLTEXT_FV_NAME
)

os.environ["HOPSWORKS_API_KEY"] = "1QJZ515qO3Hl6pwr.Kr6HwXJ5SbnYV6TeEyAEyDGsV31Is9rryhZUyvRjamJjodvONIodYhBskNcZxHAz"

## <span style="color:#ff5f27">🧬 Metadata and Text Extraction, Embedding Creation </span>

In [ ]:
# === Cell 3: Parse Zotero RDF ===

parser = ZoteroRDFParser(
    rdf_file_path=RDF_PATH,
    base_attachment_dir=BASE_DIR
)

papers = parser.parse()

print(f"Parsed {len(papers)} papers.")
papers[:2]  # 快速 sanity check


In [ ]:
# === Cell 4: Extract Full Text from Attachments ===

metadata_rows = []
fulltext_rows = []

for paper in papers:
    full_text = ""

    for attach in paper["attachments"]:
        full_path = os.path.join(BASE_DIR, attach["path"])
        content = ContentProcessor.read_file(full_path, attach["type"])

        if len(content) > MIN_FULLTEXT_LEN:
            full_text = content
            break  # 只取一个可读正文

    # Zotero 没摘要 → 从全文兜底
    if not paper["abstract"] and full_text:
        fallback_abs = ContentProcessor.extract_abstract_fallback(full_text)
        if fallback_abs:
            paper["abstract"] = fallback_abs

    # --- Metadata Row ---
    metadata_rows.append({
        "paper_id": paper["id"],
        "title": paper["title"],
        "abstract": paper["abstract"],
        "authors": paper["authors"],
        "year": paper["year"],
        "category": paper["category"],
        "combined_text": (
            f"Title: {paper['title']}\n"
            f"Category: {paper['category']}\n"
            f"Abstract: {paper['abstract']}"
        )
    })

    # --- Full-text Rows ---
    if full_text:
        chunks = ContentProcessor.chunk_text(full_text)
        for i, chunk in enumerate(chunks):
            fulltext_rows.append({
                "paper_id": paper["id"],
                "chunk_index": i,
                "content": chunk,
                "year": paper["year"]
            })

print(f"Metadata rows: {len(metadata_rows)}")
print(f"Fulltext chunks: {len(fulltext_rows)}")


## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks
from config import HOPSWORKS_PROJECT, HOPSWORKS_API_KEY
# project = hopsworks.login()

project = hopsworks.login(
        project=HOPSWORKS_PROJECT,
        api_key_value=HOPSWORKS_API_KEY
    )

fs = project.get_feature_store()

## <span style="color:#ff5f27;"> 🪄 Feature Group Creation </span>

In [ ]:
from hsfs import embedding

# # Create the Embedding Index
# emb = embedding.EmbeddingIndex()

# emb.add_embedding(
#     "embeddings",
#     model.get_sentence_embedding_dimension(),
# )

# # Get or create the 'documents_fg' feature group
# documents_fg = fs.get_or_create_feature_group(
#     name="documents_fg",
#     description='Information from various files, presenting details like file names, source links, and structured text excerpts from different pages and paragraphs.',
#     version=1,
#     primary_key=['context_id'],
#     online_enabled=True,
#     embedding_index=emb,
# )

# documents_fg.insert(df_text_processed)
model = SentenceTransformer(MODEL_NAME)

df_meta = pd.DataFrame(metadata_rows)
df_meta["embedding"] = df_meta["combined_text"].apply(
    lambda x: model.encode(x).tolist()
)

meta_fg = fs.get_or_create_feature_group(
    name="zotero_meta_fg",
    version=1,
    description="Paper Metadata (Title + Abstract)",
    primary_key=["paper_id"],
    online_enabled=True,
    embedding_index=EmbeddingIndex()
)

meta_fg.insert(df_meta)

print("✅ Metadata Feature Group uploaded.")

In [ ]:
if fulltext_rows:
    df_text = pd.DataFrame(fulltext_rows)
    df_text["embedding"] = df_text["content"].apply(
        lambda x: model.encode(x).tolist()
    )

    text_fg = fs.get_or_create_feature_group(
        name="zotero_fulltext_fg",
        version=1,
        description="Paper Full-text Chunks",
        primary_key=["paper_id", "chunk_index"],
        online_enabled=True,
        embedding_index=EmbeddingIndex()
    )

    text_fg.insert(df_text)
    print("✅ Full-text Feature Group uploaded.")
else:
    print("⚠️ No full-text chunks found.")


## <span style="color:#ff5f27;">🪄 Feature View Creation </span>


In [ ]:
# # Get or create the 'documents' feature view
# feature_view = fs.get_or_create_feature_view(
#     name="documents",
#     version=1,
#     description='Chunked context for RAG system',
#     query=documents_fg.select(["file_name", "file_link", "page_number", "paragraph", "text"]),
# )

In [ ]:
# === Feature View: Paper-level Metadata ===

paper_meta_fv = fs.get_or_create_feature_view(
    name="zotero_papers",
    version=1,
    description="Paper-level metadata for literature search and selection",
    query=meta_fg.select([
        "paper_id",
        "title",
        "abstract",
        "authors",
        "year",
        "category"
    ])
)


In [ ]:
# === Feature View: Full-text Chunks for RAG ===

paper_chunks_fv = fs.get_or_create_feature_view(
    name="zotero_chunks",
    version=1,
    description="Chunked full-text context for RAG and paper reading agent",
    query=text_fg.select([
        "paper_id",
        "chunk_index",
        "content",
        "year"
    ])
)


---